In [ ]:
import numpy as np
import pandas as pd


In [ ]:
#Definición de las variables.py del juego
DIMENSIONES = 10  # Tamaño del tablero (10x10)
BARCOS = {
    "portaaviones": 4,  # Barco de 4 posiciones
    "submarino": 3,     # Dos barcos de 3 posiciones
    "destructor": 2,    # Tres barcos de 2 posiciones
    "patrullero": 1     # Cuatro barcos de 1 posición
}
CARACTER_AGUA = "~"
CARACTER_BARCO = "B"
CARACTER_IMPACTO = "X"
CARACTER_FALLO = "O"

In [ ]:
# funciones.py
import numpy as np
from variables import DIMENSIONES, CARACTER_AGUA, CARACTER_BARCO, CARACTER_IMPACTO, CARACTER_FALLO

def generar_coordenadas_aleatorias():
    """
    Genera y devuelve una coordenada aleatoria (x, y) dentro de los límites del tablero.
    """
    x = np.random.randint(0, DIMENSIONES)
    y = np.random.randint(0, DIMENSIONES)
    return x, y

def colocar_barco(tablero, eslora):
    """
    Coloca un barco de longitud 'eslora' en el tablero de forma aleatoria.
    Se asegura de que el barco no salga del tablero ni se superponga con otro barco.
    """
    while True:
        orientacion = np.random.choice(['H', 'V'])  # Horizontal o Vertical
        x, y = generar_coordenadas_aleatorias()

        if orientacion == 'H' and y + eslora <= DIMENSIONES:
            # Comprobamos que no haya barcos en el área seleccionada
            if np.all(tablero[x, y:y+eslora] == CARACTER_AGUA):
                tablero[x, y:y+eslora] = CARACTER_BARCO
                return True
        elif orientacion == 'V' and x + eslora <= DIMENSIONES:
            # Comprobamos que no haya barcos en el área seleccionada
            if np.all(tablero[x:x+eslora, y] == CARACTER_AGUA):
                tablero[x:x+eslora, y] = CARACTER_BARCO
                return True

def inicializar_barcos(tablero, barcos):
    """
    Inicializa todos los barcos en el tablero dado.
    'barcos' es un diccionario con el tipo de barco y su eslora.
    """
    for tipo, eslora in barcos.items():
        for _ in range(barcos[tipo]):
            colocar_barco(tablero, eslora)

def imprimir_tablero(tablero, visible=False):
    """
    Imprime el tablero en un formato de cuadrícula en la consola.
    - `tablero`: El tablero a imprimir (array de numpy).
    - `visible`: Si es True, oculta la ubicación de los barcos que no han sido impactados.
    """
    print("   " + " ".join(str(i) for i in range(DIMENSIONES)))  
    print("  " + "-" * (DIMENSIONES * 2))  # Separador de cabecera
    for i in range(DIMENSIONES):
        # Fila con índices y contenido del tablero
        fila = []
        for j in range(DIMENSIONES):
            # Mostrar solo los impactos y fallos si el tablero es el visible del jugador
            if visible and tablero[i, j] == CARACTER_BARCO:
                fila.append(CARACTER_AGUA)  
            else:
                fila.append(tablero[i, j])  
        print(f"{i} | " + " ".join(fila))
    print()  # Línea de separación

def obtener_coordenadas_usuario():
    """
    Solicita al usuario una coordenada para disparar y la valida.
    """
    while True:
        try:
            x, y = map(int, input("Introduce las coordenadas de disparo (x y): ").split())
            if 0 <= x < DIMENSIONES and 0 <= y < DIMENSIONES:
                return x, y
            else:
                print(f"Por favor, introduce coordenadas dentro del rango (0-{DIMENSIONES-1}).")
        except ValueError:
            print("Entrada inválida. Introduce dos números separados por un espacio.")

In [ ]:
# tablero.py
import numpy as np
from variables import DIMENSIONES, CARACTER_AGUA, CARACTER_BARCO, CARACTER_IMPACTO, CARACTER_FALLO, BARCOS

class Tablero:
    def __init__(self, id_jugador):
        """
        Constructor para la clase Tablero
        - `id_jugador`: identifica a qué jugador pertenece el tablero (jugador o máquina)
        """
        self.id_jugador = id_jugador
        self.dimensiones = DIMENSIONES  
        # Crear dos tableros: uno visible para el jugador y otro interno para la máquina
        self.tablero_interno = np.full((DIMENSIONES, DIMENSIONES), CARACTER_AGUA)  
        self.tablero_visible = np.full((DIMENSIONES, DIMENSIONES), CARACTER_AGUA)  
        self.barcos = BARCOS  
        self.colocar_barcos_iniciales()

    def colocar_barcos_iniciales(self):
        """
        Coloca todos los barcos en el tablero de acuerdo a los tipos de barcos definidos en `BARCOS`
        """
        for tipo, eslora in self.barcos.items():
            # Colocar la cantidad de barcos de cada tipo en el tablero
            for _ in range(self.barcos[tipo]):
                self._colocar_barco(eslora)

    def _colocar_barco(self, eslora):
        """
        Intenta colocar un barco de longitud 'eslora' de forma aleatoria en el tablero.
        """
        while True:
            # Generamos la orientación y las coordenadas aleatorias
            orientacion = np.random.choice(['H', 'V'])  
            x, y = self._generar_coordenadas_aleatorias()

            if orientacion == 'H' and y + eslora <= DIMENSIONES:
                # Comprobamos que no haya barcos en la fila horizontal seleccionada
                if np.all(self.tablero_interno[x, y:y+eslora] == CARACTER_AGUA):
                    self.tablero_interno[x, y:y+eslora] = CARACTER_BARCO
                    return
            elif orientacion == 'V' and x + eslora <= DIMENSIONES:
                # Comprobamos que no haya barcos en la columna vertical seleccionada
                if np.all(self.tablero_interno[x:x+eslora, y] == CARACTER_AGUA):
                    self.tablero_interno[x:x+eslora, y] = CARACTER_BARCO
                    return

    def _generar_coordenadas_aleatorias(self):
        """
        Genera coordenadas aleatorias (x, y) dentro de los límites del tablero.
        """
        x = np.random.randint(0, self.dimensiones)
        y = np.random.randint(0, self.dimensiones)
        return x, y

    def imprimir_tablero(self, visible=False):
        """
        Imprime el tablero en la consola. Si 'visible' es True, muestra el tablero visible para el jugador.
        De lo contrario, muestra el tablero interno para el jugador.
        """
        tablero = self.tablero_visible if visible else self.tablero_interno
        print("   " + " ".join(str(i) for i in range(self.dimensiones)))
        print("  " + "-" * (self.dimensiones * 2))
        for i, fila in enumerate(tablero):
            print(f"{i} | " + " ".join(fila))
        print()

    def recibir_disparo(self, x, y):
        """
        Recibe un disparo en la coordenada (x, y) y actualiza el tablero según el resultado.
        - Si hay un barco, marca el impacto.
        - Si es agua, marca el fallo.
        """
        if self.tablero_interno[x, y] == CARACTER_BARCO:
            self.tablero_interno[x, y] = CARACTER_IMPACTO
            self.tablero_visible[x, y] = CARACTER_IMPACTO
            return True  # Impacto
        else:
            self.tablero_visible[x, y] = CARACTER_FALLO
            return False  # Agua

    def barcos_restantes(self):
        """
        Devuelve la cantidad de posiciones de barco sin impactar en el tablero.
        """
        return np.sum(self.tablero_interno == CARACTER_BARCO)

print(Tablero)


In [ ]:
#Antes de empezar el juego
## Vidas
num_vidas_h = 20
num_vidas_m = 20

regla_juego = """Bienvenid@ a hundir la flota! Tu contrincante va a ser una máquina. Cada uno disparará a una coordenada.
Si logras disparar en una posición de un barco de la maquina, aparecerá una 'X' en el tablero del 
adversario y podrás seguir disparando vayas acertando. Si, por el contrario, disparas al agua, apracerá un '-' en 
eltablero del adversario y el turno será de la maquina. Para ganar, se tienen que "hundir" todos los barcos del adversario."""

# Definir los barcos con sus respectivas longitudes
barcos = {         "barco1": 1, 
                   "barco2": 1, 
                   "barco3": 1, 
                   "barco4": 1, 
                   "barco5": 2, 
                   "barco6": 2, 
                   "barco7": 2, 
                   "barco8": 3, 
                   "barco9": 3, 
                   "barco10": 4}


In [ ]:
#Juego final main.py

import numpy as np
from tablero import Tablero
from funciones import generar_coordenadas_aleatorias, imprimir_tablero
from variables import DIMENSIONES
from variables import BARCOS

def empezar_juego():
    pregunta_01 = input("Hola, ¿quieres iniciar una partida?(Contesta si o no)")

    if pregunta_01 == "si":
        pregunta_02 = input("¿Cómo te llamas?")
        Tablero.jugador = pregunta_02
        Tablero.maquina = "maquina"

        print(regla_juego)

        posicion_barco(Tablero.tablero_jugador_barcos, Tablero.filas, Tablero.columnas)
        posicion_barco(Tablero.tablero_maquina_barcos, Tablero.filas, Tablero.columnas)
        print("*"*40)
        print("Tablero de", pregunta_02)
        print("*"*40)
        print(Tablero.tablero_jugador_barcos)
        print("*"*40)
        print("Tablero del", Tablero.maquina)
        print("*"*40)
        print(Tablero.tablero_maquina_nada)
    else:
        print("¡Hasta luego!")

empezar_juego()



def main():
    print("¡Bienvenido a Hundir la Flota!")
    print("Instrucciones: Intenta hundir todos los barcos de la máquina antes de que hundan los tuyos.")
    
    # Inicializar tableros de jugador y máquina
    tablero_jugador = Tablero(id_jugador="jugador")
    tablero_maquina = Tablero(id_jugador="máquina")

    juego_terminado = False
    turno_jugador = True  

    while not juego_terminado:
        if turno_jugador:
            print("\nTu tablero de disparos:")
            imprimir_tablero(tablero_maquina.tablero_visible)
            x, y = map(int, input("Introduce las coordenadas de disparo (x y): ").split())
            
            if tablero_maquina.recibir_disparo(x, y):
                print("¡Impacto! Vuelve a disparar.")
            else:
                print("Agua... le toca a la máquina.")
                turno_jugador = False
        else:
            x, y = generar_coordenadas_aleatorias()
            print(f"La máquina dispara a ({x}, {y})...")
            
            if tablero_jugador.recibir_disparo(x, y):
                print("¡La máquina te ha dado! Vuelve a disparar.")
            else:
                print("La máquina falló... ¡Tu turno!")
                turno_jugador = True
            
        # Comprobación de fin de juego
        if tablero_maquina.barcos_restantes() == 0:
            print("¡Felicidades! Has hundido todos los barcos enemigos. ¡Ganaste!")
            juego_terminado = True
        elif tablero_jugador.barcos_restantes() == 0:
            print("La máquina ha hundido todos tus barcos. Has perdido.")
            juego_terminado = True

if __name__ == "__main__":
    main()